RAG(Retrieval-Augmented Generation)

It combines two separate AI capabilities:

->Retrieval → Finding relevant information from a knowledge base (like your PDF, FAISS, or database)

->Generation → Using a language model (like Llama 3.1, GPT, etc.) to generate a natural language answer

So instead of relying only on the model’s internal memory or training data (which might be outdated or incomplete),
the model is augmented with external retrieved knowledge — that’s why we call it Retrieval-Augmented Generation.

Without RAG vs With RAG

1.Without RAG (Pure Generation)

If you just ask:

“Summarize the main highlights of the 2024 Indian budget.” to a plain LLM (like Llama or GPT),
the model answers from its training data, which:

->Might be outdated

->Might not contain your specific document

->Might hallucinate (make up info)

So the model guesses rather than knowing.

2.With RAG (Retrieval-Augmented Generation)

When you use RAG, the pipeline does this:

->Retrieve relevant text from external sources (like your PDF → FAISS)

->Augment the user question with that retrieved text (context)

->Generate the answer using the model — based only on that retrieved context

That’s why it’s called Retrieval + Augmentation + Generation.

In short:

The model doesn’t guess anymore — it reads the right info first, then answers.

In [1]:
import os
from pathlib import Path

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import OllamaLLM


In [5]:
PDF_PATH = r"C:\Users\ANIGETH\Downloads\Langchain_project\pdf_query_ollama\budget_speech.pdf"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"   # 384-dim
OLLAMA_MODEL = "llama3.1"                                # already installed via `ollama pull llama3.1`
INDEX_DIR = "./faiss_index"                               # folder to persist FAISS
CHUNK_SIZE = 800
CHUNK_OVERLAP = 50
TOP_K = 3



In [6]:
# Loadinf PDF 
assert Path(PDF_PATH).exists(), f"PDF not found at: {PDF_PATH}"
loader = PyPDFLoader(PDF_PATH)
docs = loader.load()
print(f"📄 Loaded {len(docs)} pages")



📄 Loaded 60 pages


In [7]:
# Splitting into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)
documents = splitter.split_documents(docs)
print(f"✂️ Split into {len(documents)} chunks")



✂️ Split into 146 chunks


In [10]:
#Embedding
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)



c:\Users\ANIGETH\Downloads\Langchain_project\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Load FAISS(Facebook AI Similarity Search)index
if not Path(INDEX_DIR).exists():
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(INDEX_DIR)
    print(f"🆕 Built FAISS index at {INDEX_DIR}")
else:
    vectorstore = FAISS.load_local(INDEX_DIR, embeddings, allow_dangerous_deserialization=True)
    print(f"📦 Loaded existing FAISS index from {INDEX_DIR}")

retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})



🆕 Built FAISS index at ./faiss_index


In [12]:
 # LLM (LOCAL OLLAMA)
llm = OllamaLLM(model=OLLAMA_MODEL)
parser = StrOutputParser()

# Prompt: you can tweak tone/formatting here
prompt = ChatPromptTemplate.from_template(
"""You are a helpful assistant. Use ONLY the context to answer clearly and concisely.
If the answer is not in the context, say "I don't know from the provided document."

Context:
{context}

Question:
{question}

Answer:"""
)



In [13]:
def ask(query: str) -> str:
    """Retrieve relevant chunks from FAISS and answer using Llama (Ollama)."""
    retrieved = retriever.invoke(query)
    context = "\n\n".join(d.page_content for d in retrieved)
    msg = prompt.format(context=context, question=query)
    # OllamaLLM takes a string; returns a string
    raw = llm.invoke(str(msg))
    return parser.invoke(raw)



In [14]:
question = "Summarize the main highlights of the budget speech"
answer = ask(question)
print("\n Answer:\n", answer)



 Answer:
 The main highlights of the Budget Speech are:

1. Accelerate growth
2. Secure inclusive development
3. Invigorate private sector investments
4. Uplift household sentiments
5. Enhance spending power of India's rising middle class
6. Transformative work done in first two terms guides the government to march forward


In [15]:
question = "Maaking any invetsment on AI"
answer = ask(question)
print("\n Answer:\n", answer)



 Answer:
 A Centre of Excellence in Artificial Intelligence (AI) for education will be set up with a total outlay of ` 500 crore.
